In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import get_custom_objects
import scipy.io
from keras import backend as K
from keras.layers import Activation

In [2]:
# define the filename to save the weights of the model as a .mat
filename = 'models/hopf_critical_1'

# define the subset of the (x_1,x_2)-space to approximate dynamics
lower_limit = 3.5
upper_limit = 6.5
step_size = 0.1;

# hardcode the gamma, beta, tau and  constants in the chemical perceptron (later examples automatically train this parameter)
# Note -  tau corresponds to the scaling of  -tau*y^2 (see Appendix B - (32) & (33))
gamma = 1
beta_1 = 1
tau = 1;

# define the number of chemical perceptrons species in a single hidden layer
N = 6;

# bifurication center and critical point (we have shifted the normal form of the Hopf bifurcation in Cartesian coordinates)
a = 5;

# define the number of epochs to use in training the model
number_of_epochs = 1

In [3]:
# define the custom activation function for the training with particular values of gamma and tau 
def smooth_max_activation(x):
    return 0.5*(x + K.sqrt(K.square(x)+4*gamma*tau))/tau

get_custom_objects().update({'smooth_max_activation': Activation(smooth_max_activation)})


# define training for the executive species and parameter species
x1_train = np.arange(lower_limit, upper_limit, step_size, dtype="float32")
x2_train = np.arange(lower_limit, upper_limit, step_size, dtype="float32")
x1v, x2v = np.meshgrid(x1_train, x2_train, indexing='ij')
nx1 = len(x1_train)
nx2 = len(x2_train)


# compute the target ODE i.e. the normal form of a Hopf bifurcation in Cartesian coordinates shifted into the positive orthant with its parameter set to zero
critical_param = 0
D = critical_param -np.power(x1v-a, 2) -np.power(x2v-a, 2)
g_1 = D*(x1v-a) - (x2v-a)
g_2 = D*(x2v-a) + (x1v-a)

# according to Algorithm 1 in https://doi.org/10.48550/arXiv.2406.03456
y1_train = (g_1 - beta_1)/x1v
y2_train = (g_2 - beta_1)/x2v

# process the array shapes 
x_train = np.append(x1v.reshape(-1,1), x2v.reshape(-1,1), axis=1)
y_train = np.append(y1_train.reshape(-1,1), y2_train.reshape(-1,1), axis=1)

In [4]:
# define the quasi-static approxiamtion according to Algorithm 1 in https://doi.org/10.48550/arXiv.2406.03456
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(2,), name=''),
  tf.keras.layers.Dense(N, activation='smooth_max_activation'),
  tf.keras.layers.Dense(2, activation=None, use_bias=False),
])

# compile model with an optimizer and mse loss function
model.compile(optimizer='adam',
              loss='mse',
              metrics=['mse'])

In [5]:
# train the quasi-static neural network model to replicate x_train -> y_train
model.fit(x_train, y_train, epochs=number_of_epochs, validation_split = 0.1)

26/26 [==============================] - 1s 7ms/step - loss: 0.3538 - mse: 0.3538 - val_loss: 0.4133 - val_mse: 0.4133


In [6]:
# view the model structure, each 'Param' corresponds to at least one rate of reaction in the chemical system
model.summary()

# save the weights of this neural network for use in ODE simulations in MATLAB
first_layer_weights = model.layers[1].get_weights()[0]
first_layer_biases = model.layers[1].get_weights()[1]
output_layer_weights = model.layers[2].get_weights()[0]

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
  (Flatten)                  (None, 2)                 0         
                                                                 
 dense (Dense)               (None, 6)                 18        
                                                                 
 dense_1 (Dense)             (None, 2)                 12        
                                                                 
Total params: 30 (120.00 Byte)
Trainable params: 30 (120.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [7]:
scipy.io.savemat(filename+'.mat', {'first_layer_weights':first_layer_weights, 
                                   'first_layer_biases':first_layer_biases, 
                                   'output_layer_weights':output_layer_weights,
                                   'gamma': gamma,
                                   'beta': beta_1,
                                   'alpha': tau #abuse of notation - still refering to tau  
                                  })